<a name="top"></a>
## Finding Bad Joins 
jcl 2023-12-17

It's a fair assumption that a first draft of any sql script will contain bad joins resulting from misunderstanding the effective grain of tables or paste error typos. So it's worthwhile developing a system to catch those before flooding your network with data till you crash your notebook with a memory error. Here we show an example of typical errors, and how we code to catch them with minimal trouble. <p>
- [populating sqlite3](#pop)
- [bad joins](#badj)
- [bad report](#badr)
- [fixed join](#fixed)

In [1]:
import pandas as pd
import sqlite3
from pygments import highlight
from pygments.lexers import SqlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML

In [2]:
conn = sqlite3.connect("badJoin.db")

<a name="pop"></a>
### Populating SQLITE3 
[top](#top)

In [3]:
pencilD = {"pencil_id": range(1, 6) \
           , "brand": "Crayola Dixon BIC Crayola Crayola".split()}
xrefD = {"xref_id": range(1, 11), "xref_test_id": range(1, 11) \
         , "xref_pencil_id": [1, 2, 3, 4, 5, 3, 2, 3, 2, 1]}
testD = {"test_id": range(1, 11) \
         , "condition": "new dull ok ok new dull ok dull ok ok".split()}
pencilR, xrefR, testR = [pd.DataFrame(xD) for xD in (pencilD, xrefD, testD)]
list(map(len, (pencilR, xrefR, testR)))

[5, 10, 10]

In [4]:
for xR, tn in zip((pencilR, xrefR, testR), "pencil xref test".split()):
    xR.to_sql(tn, conn)

ValueError: Table 'pencil' already exists.

<a name="badj"></a>
### Bad Joins
There are bad joins here. Joining the pencil to the xref table will cause multiple matches, which might be ok, but might be a surprise. Joining the cross cte to the test table by a just wrong on condition will cause an outer join. The checker cte illustrates the situation. By selecting the checker cte for {final}, we cna have a cell that shows us the situation.
[top](#top)

In [ ]:
templateS = open("badJoin.sql").read()
display(HTML(data=highlight(templateS, SqlLexer(), HtmlFormatter())))

<a name="badr"></a>
### Bad Report
First we notice there is one more match to xref than Crayola branded pencils. Second, there is an insane number of tests. 
[top](#top)

In [ ]:
pd.read_sql(templateS.format(brand="Crayola", final="checker"), conn) \
    .sort_values("cten")

<a name="fixed"></a>
### Fixed Code
[top](#top)

In [5]:
templateS = open("badJoinFixed.sql").read()
display(HTML(data=highlight(templateS, SqlLexer(), HtmlFormatter())))

<a name="goodr"></a>
### Good Report
The number of tests matches the number of branded pencils. 
[top](#top)

In [6]:
pd.read_sql(templateS.format(brand="Crayola", final="checker"), conn) \
    .sort_values("cten")

,cten,n
0,0 pencils,5
1,1 branded,3
2,2 matched to xref,3
3,3 matched to tests,3


### Getting Data
Now we are confident we are not going to flood ourselves with outer joins, we can change final from checker to getter, grab our dataframe and run a report. 

In [7]:
crayolaR = pd.read_sql(templateS.format(brand="Crayola", final="getter"), conn)
crayolaR["condition"].value_counts().sort_index()

condition
new    1
ok     2
Name: count, dtype: int64